# Initialize database files

In [ ]:
%cd twitteranalysis
import environment
import sqlite3
import os
from Loggers.Helpers import delete_files


g = environment.sqlite_file_connection_string_generator()
query = """CREATE TABLE `word_map_deux` (
`tweet_id` int(20) DEFAULT NULL,
  `user_id` int(20) DEFAULT NULL,
  `word` varchar(200) DEFAULT NULL,
  `sentence_index` int(11) DEFAULT NULL,
  `word_index` int(11) DEFAULT NULL)"""

# Make master db and tables
import sqlite3

# make individual tables
def initialize_working_tables():
    for i in range(0, environment.MAX_DB_FILES +1):
        try:
            f = '%s/wordmapping%s.db' % (environment.DB_FOLDER, i)
            conn = sqlite3.connect(f)
            conn.execute(query)
            conn.commit()
            conn.close()
        except:
            pass

def initialize_master_db():
    try:
        conn = sqlite3.connect(environment.MASTER_DB)
        conn.execute(query)
        conn.commit()
        conn.close()
    except:
        pass

def delete_master_db():
    os.remove(environment.MASTER_DB)
    
def delete_working_files():
    delete_files(environment.DB_FOLDER)

In [ ]:
delete_master_db()
initialize_master_db()

In [ ]:
delete_working_files()
initialize_working_tables()

# Run the user parser

In [ ]:
%cd twitteranalysis
%run -i Executables/process_user_descriptions_into_words2.py

In [ ]:
class AsyncBatcher(object):
    __slots__ = ['batch', 'batch_size', 'save', 'flush']
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.batch = []
    
    

In [ ]:
%cd twitteranalysis
from Executables.process_user_descriptions_into_words2 import main
main()


In [ ]:
%cd twitteranalysis
from Executables.process_user_descriptions_into_words2 import main
main()


# Clear queue

In [ ]:
%cd twitteranalysis

from Servers.ClientSide import Client
c = Client()

# Each of the listening request handlers needs its queue flushed
c.send_flush_command()

#j = c.send_shutdown_command()

In [ ]:
j

# Merge the various sqlite files

In [ ]:

%cd twitteranalysis
import environment
import sqlite3

mainDB = environment.MASTER_DB
columnNames = "word_index, sentence_index, word, user_id, tweet_id"
    
otherDbNames = [ 'wordmapping%s.db' % (i) 
            for i in range( 0, environment.MAX_DB_FILES +1)]

otherDBs = [ '%s/%s' % (environment.DB_FOLDER, name) 
            for name in otherDbNames]
    
for db in otherDbNames:
    conn = sqlite3.connect( mainDB )  # Connect to the main database
    curs = conn.cursor()  # Connect a cursor
    
    dbPath = '%s/%s' % (environment.DB_FOLDER, db)
    curs.execute( "ATTACH DATABASE ? as ? ;", (dbPath, 'db') )

    dbNameTableName = "db.word_map_deux"
    tableName = 'word_map_deux'
    curs.execute( "INSERT INTO %s (%s) SELECT %s FROM %s;" % (tableName, columnNames, columnNames, dbNameTableName) )
    conn.commit()
    curs.close()
    conn.close()


# Check data

In [ ]:
%cd twitteranalysis
import environment
import sqlite3

In [ ]:
uids = []
rows = []
print("Unique user ids; rows")
for db in otherDbNames:
    dbPath = '%s/%s' % (environment.DB_FOLDER, db)
    conn = sqlite3.connect( dbPath )  # Connect to the main database
    curs = conn.cursor()  # Connect a cursor
    r1 = conn.execute("select count( distinct user_id) from word_map_deux")
    v =  r1.fetchone()
    uids.append(v[0])
    r2 = conn.execute("select count( word) from word_map_deux")
    v2 =  r2.fetchone()
    rows.append(v2[0])

    print("%s : %s; %s " % (db, v[0], v2[0]))
    conn.close()


In [ ]:
sum(uids)

In [ ]:
sum(rows)

In [ ]:
conn = sqlite3.connect(environment.MASTER_DB)
r1 = conn.execute("select count( distinct user_id) from word_map_deux")
print("%s Unique user ids" % r1.fetchone())
r2 = conn.execute("select count( word) from word_map_deux")
print("%s rows in master.db" % r2.fetchone())
conn.close()

NB: 
    - numberUsers processed = 4352
    - users w empty descriptions = 332
    - users non-english = 129
    - expected users: 3891

5/15 1.36
    - lock added to sqlite writer
    - 4085 unique ids based on files
    - 3950 unique ids based on master
    - 67002 rows in master and based on files


5/15 11.12
    - 4059 unique ids based on files
    - 3923 unique ids based on master
    - 66507 rows in master and based on files

5/15 10.39
    - 3653 unique ids based on files
    - 3529 unique ids based on master
    - 60060 rows in master and based on files
    

5/14 9.00
    - 4014 unique ids based on files
    - 2510 unique ids based on master
    - 41973 rows in master and based on files


Before changed to class method w separate call to flush
    - 3591 Unique user ids
    - 47952 rows in master.db

After
    - 4014 Unique user ids
    - 67940 rows in master.db
    



In [ ]:
likelyNonEmpty = 4352 - 332 - 129
likelyNonEmpty

In [9]:
3950 + 332

4282

# Figure out what users are missing

In [1]:
%cd twitteranalysis
import sqlite3
import environment
import DataTools.Cursors
cursor = DataTools.Cursors.WindowedUserCursor( language='en' )


(bookmark:twitteranalysis) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
connection ready


In [2]:
conn = sqlite3.connect(environment.MASTER_DB)
curs = conn.cursor()  # Connect a cursor
    
missing = []

while True:
    user = cursor.next()
    q = "select * from word_map_deux where user_id = %s" % user.userID
    r1 = curs.execute(q)
    r = r1.fetchone()
    if r is None:
        missing.append(user.userID)


curs.close()
conn.close()
print(len(missing))

StopIteration: 

In [7]:
import pandas as pd
from DataTools.DataConnections import MySqlConnection, DAO
conn = MySqlConnection(environment.CREDENTIAL_FILE)
conn._make_engine()

In [13]:
descripts = []
for userId in missing:
    q = 'select description from users where userID = %s' % userId
    v = pd.read_sql_query(q, conn.engine).iloc[0].values[0]
    descripts.append((userId, v))
len(descripts)

402

In [16]:
substantive = [x for x in descripts if x[1] != '']
len(substantive)

70

In [17]:
substantive

[(168391959, '**SKYHIGH**'),
 (401742354, '✌'),
 (452749712, ':)'),
 (1170228427, '|1104|•|0717|'),
 (1268582460, '☀♑ · ☾♑ · ☿♐ · ♀♒ · ♂♉ · ♃♌ · ♄♑ · ♅♑ · ♆♑ · ♇♏'),
 (1725045786,
  'Mature-age,Disabled Uni Student. Member of the ALP. Founding Friend and a Soc/Med Ambassador for the Climate Council.'),
 (1725884737,
  'Her View From Omaha. Real. Local. Lifestyle. We strive to highlight your view each day.'),
 (1726692926,
  'A provider of home medical equipment, daily living aids, bath safety, electrotherapy stimulators, and many other pain relieving products for chronic/acute pain.'),
 (1728794102,
  "I am a #SNHU student in love with #webdevelopment, #coding, #hacking & #socialmedia. I'm a #WOC #Lesbian #Pagan & #Nerd with multiple tattoos."),
 (1731736086,
  'They Call Me High-Functioning: A Memoir of Rape, Mental Illness & Bad Mothering. Read excerpts at my blog.'),
 (1732483676, '[ ???¿¿¿???¿¿¿??? ]'),
 (1733236621,
  'Our mission is to empower & promote healthier lifestyles to th

In [ ]:
conn = sqlite3.connect(environment.MASTER_DB)
r1 = conn.execute("select user_id, count(word) from word_map_deux group by user_id ")
print( r1.fetchone())
conn.close()

In [ ]:
conn = sqlite3.connect(environment.MASTER_DB)
curs = conn.cursor()  # Connect a cursor
    
r1 = curs.execute("select * from word_map_deux where user_id = 2")
# for i in range(0, 100):
print( r1.fetchone())

curs.close()

In [ ]:
"""The above tells me that the problem is the whole store getting sent to the server"""

# Explore data

In [ ]:
def master_row_generator():
    conn = sqlite3.connect(environment.MASTER_DB)
    r = conn.execute("select * from word_map_deux")
    while True:
        yield r.fetchone()

def db_row_generator(filepath):
    conn = sqlite3.connect(filepath)
    r = conn.execute("select * from word_map_deux")
    while True:
        yield r.fetchone()


In [ ]:
gen = master_row_generator()

In [ ]:
for i in range(0, 220):
    print(next(gen))

In [ ]:
f = '/Users/adam/Desktop/TwitterDataAnalysisLogs/dbs/wordmapping1.db'
dgen = db_row_generator(f)
for i in range(0, 220):
    print(next(dgen))

In [ ]:
f = '/Users/adam/Desktop/TwitterDataAnalysisLogs/dbs/wordmapping1.db'
conn = sqlite3.connect(f)
r = conn.execute("select * from word_map_deux")
for i in range(0, 5):
    print(r.fetchone())


In [ ]:
f = '/Users/adam/Desktop/TwitterDataAnalysisLogs/dbs/wordmapping9.db'
conn = sqlite3.connect(f)
r = conn.execute("select * from word_map_deux")
for i in range(0, 5):
    print(r.fetchone())


In [ ]:

f = '/Users/adam/Desktop/TwitterDataAnalysisLogs/master.db'
conn = sqlite3.connect(f)
r = conn.execute("select count(word) from word_map_deux")
print(r.fetchone())
conn.close()

# Timing

In [ ]:
%cd twitteranalysis

from functools import wraps
import time

from Loggers.CsvLoggers import log_query, log_query_timestamp
import environment


def time_and_log(fn):
    """
    Decorator to time operation of method
    From High Performance Python, p.27
    """
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        t2 = time.time()
        elapsed = t2 - t1
        log_query( elapsed )
        log_query_timestamp()
#         print(("@timefn:%s took %s seconds" % (fn.__name__, elapsed)))
        return result
    return measure_time


In [ ]:

@time_and_log
def test():
    time.sleep(2)
    print('done')
    

In [ ]:
test()